# getting ready
安装相关依赖

step1 下载onnx模型与测试视频


In [ ]:
!gdown --fuzzy https://github.com/BabitMF/bmf/releases/download/files/models.tar.gz
!gdown --fuzzy https://github.com/BabitMF/bmf/releases/download/files/files.tar.gz
!tar xzvf models.tar.gz
!tar xzvf files.tar.gz

step2 安装BMF、onnxruntime-module

In [ ]:
!pip install BabitMF
!pip3 install onnxruntime


step3 获取BMF源码，找到demo模块，测试modules和model文件可以正常使用

In [ ]:
!git clone https://github.com/BabitMF/bmf.git

In [ ]:
!cp /content/bmf/bmf/demo/aesthetic_assessment/*.py .

In [ ]:
import bmf
import sys
import onnxruntime as ort
from module_utils import SyncModule
import aesmod_module
import onnxruntime as ort
import os.path as osp
model_dir = osp.join(osp.abspath(osp.dirname('__file__')), 'models')
aesmod_ort_model_path = osp.realpath(osp.join(model_dir, 'aes_transonnx_update3.onnx'))
print(aesmod_ort_model_path)
ort_session = ort.InferenceSession(aesmod_ort_model_path)

# source code

##aesmod_module.py


*   func get_logger()
*   func flex_resize_aesv2()
*   class Aesmod
*   class BMFAesmod


##module_utils.py


*   class SyncModule


##main.py
main program for calling bmf api and visualize output
*   func segment_decode_ticks()
*   func get_duration()



In [ ]:
!cat aesmod_module.py

In [ ]:
!cat module_utils.py

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import bmf
import cv2, os, sys

def get_duration(video_path):
    capture = cv2.VideoCapture(video_path)
    fps = capture.get(cv2.CAP_PROP_FPS)      # OpenCV2 version 2 used "CV_CAP_PROP_FPS"
    frame_count = int(capture.get(cv2.CAP_PROP_FRAME_COUNT))
    duration = frame_count / fps
    capture.release()
    return duration

def segment_decode_ticks(video_path, seg_dur=4.0, lv1_dur_thres=24.0, max_dur=1000):
    '''
        bmf module new decode duration ticks
        - 0 < Duration <= 24s, 抽帧间隔r=1, 抽帧0~24帧
        - 24s < Duration <= 600s 分片抽取, 抽帧间隔r=1, 抽帧24帧
            - 6个4s切片, 共计6x4=24帧
        - duration > 600s, 分8片抽帧r=1, 抽帧数量32帧
            - (600, inf), 8个4s切片, 共计8x4=32帧
        最大解码长度 max_dur: 1000s
    '''
    duration = get_duration(video_path)
    duration_ticks = []
    if duration < lv1_dur_thres:
        return dict()
    elif duration <= 600:  # medium duration
        seg_num = 6
        seg_intev = (duration - seg_num * seg_dur) / (seg_num - 1)
        if seg_intev < 0.5:
            duration_ticks.extend([0, duration])
        else:
            for s_i in range(seg_num):
                seg_init = s_i * (seg_dur + seg_intev)
                seg_end = seg_init + seg_dur
                duration_ticks.extend([round(seg_init, 3), round(seg_end, 3)])
    else:  # long duration
        seg_num = 8
        seg_intev = (min(duration, max_dur) - seg_num * seg_dur) / (seg_num - 1)
        for s_i in range(seg_num):
            seg_init = s_i * (seg_dur + seg_intev)
            seg_end = seg_init + seg_dur
            duration_ticks.extend([round(seg_init, 3), round(seg_end, 3)])
    return {'durations': duration_ticks}


if __name__ == "__main__":
  input_path = "files/bbb_360_20s.mp4"
  outp_path = 'res2.json'

  option = dict()
  option['output_path'] = outp_path
  print('option',option)
  duration_segs = segment_decode_ticks(input_path)
  decode_params = {'input_path': input_path, 'video_params': {'extract_frames': {'fps': 1}}}
  decode_params.update(duration_segs)
  print('decode_params',decode_params)
  # module process

  py_module_path = os.path.abspath(os.path.dirname(os.path.dirname('__file__')))
  py_entry = 'aesmod_module.BMFAesmod'
  print(py_module_path, py_entry)

  streams = bmf.graph().decode(decode_params)
  video_stream = streams['video'].module('aesmod_module',
                                        option,
                                        py_module_path,
                                        py_entry)
  video_stream.run()



In [ ]:
!cat res2.json

res2.json的aesthetic给视频打分。评分区间映射至[0, 100]，分数越高代表画面的美学水平越高;  其中在AVA数据集上好坏分类的阈值5分映射为70.5分。
在对画面的分析中，美学是一个重要的维度。对于画面客观质量相似的画面，美学维度的评价能够进一步丰富对人类感知的拟合。对于图像美感的评价综合了色彩相关（亮度/饱和度/色彩丰富度...）、构图相关（三分线构图/对称性/前景背景对比）、语义相关（主题是否明确）、画质相关（纹理是否丰富清晰）等多个维度，同时除了摄影经验之外，图像的审美质量还受到情感和个人偏好的影响，例如对不同内容类型或风格的偏好。综上实际上的预测精度与主观感受仍然相差较大。